In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
import time
import json
import pprint
from random import randint

import sqlite3


In [28]:
url2='https://auto.8891.com.tw/usedauto-infos-1398201.html'
res2 = requests.get(url2)
soup2 = BeautifulSoup(res2.text,'lxml') 

In [59]:
title=soup2.select_one('.car-type').text
re.sub(r'\s+',' ',title).strip(' ')

'BMW 3-Series 2014款 2.0L'

In [2]:
def gen_headers():
    referers = ['tw.yahoo.com', 'www.google.com', 'http://www.msn.com/zh-tw/', 'http://www.pchome.com.tw/']
    user_agents = ['Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9',
              'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
              'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)']
    headers = {'user-agent': user_agents[randint(0, len(user_agents) - 1)],
               'referer': referers[randint(0, len(referers) - 1)]}
    return headers
gen_headers()

{'referer': 'http://www.pchome.com.tw/',
 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'}

In [4]:
conn = sqlite3.connect('/home/ubuntu/SQLiteDB/testCar.db')
cursor = conn.cursor()
# cursor.execute('''CREATE TABLE cars0626(source text, url text, title text, brand text, model text, doors text, color text, gasoline text,
#              cc INTEGER , transmission text, equip text, mileage INTEGER , years INTEGER , location text, 
#              posttime INTEGER, price NUMERIC, certificate text, wd text)''')

In [8]:
def get_car_infos(fileIn,fileOut,conn,cursor,headers):
    with open(fileIn,'r') as f:
        data = f.readlines()
   
    for line in data:
        
        carDic={}
        infos=line.split('\n')[0].split('|')
        carDic=json.loads(infos[2])
        url =infos[0]

        count = 0
        
        while count<3:
            soup=''
            try:
                
                res = requests.get(url, headers=headers)
                soup = BeautifulSoup(res.text,'lxml')
                title=soup.select_one('.car-type').text
                big_title=re.sub(r'\s+',' ',title).strip(' ')
                sub_title=carDic['title']
                carDic['title']='{} | {}'.format(big_title, sub_title)
                carDic['color'] = soup.select('.right-info .car-info ul li span')[7].text.strip('色')
                carDic['doors'] = soup.select('.car-detail-equipment .car-detail-base span')[1].text.split("：")[1]
                carDic['gasoline'] = soup.select('.car-detail-equipment .car-detail-base span')[2].text.split("：")[1].strip('車')
                carDic['wd'] = soup.select('.car-detail-equipment .car-detail-base span')[4].text.split("：")[1]
                if soup.select('div.tip > a'):
                    carDic['certificate'] = '其他'
                else:
                    carDic['certificate'] = 'na'
                equip=''
                for item in soup.select('.car-equipment-show .info'):
                    equip += '{}|'.format(item.text)
                carDic['equip'] = equip.strip('|')

                with open(fileOut,'a') as f:
                    f.write('{}\n'.format(json.dumps(carDic, ensure_ascii=False)))
                
                cursor.execute('INSERT INTO cars0626 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                (carDic['source'], carDic['url'] , carDic['title'], carDic['brand'], carDic['model'], carDic['doors'], 
                carDic['color'], carDic['gasoline'], carDic['cc'], carDic['transmission'], carDic['equip'],
                carDic['mileage'], carDic['years'], carDic['location'], carDic['posttime'], 
                carDic['price'], carDic['certificate'], carDic['wd']))
                conn.commit()            
                
                break
                
            except IndexError as e:
                try:
                    carDic['color'] = soup.select('.mb-info span')[3].text.strip('色')
                    carDic['gasoline'] = soup.select('.auto_standard span')[1].text.strip('車')
                    carDic['wd'] = soup.select('.auto_standard span')[2].text
                    carDic['doors'] = soup.select('.auto_standard span')[4].text
                    certi_list=['sum','save','hot']
                    if soup.select_one('.auto-check-t'):
                        text =soup.select_one('.auto-check-t').text.lower()
                        keyword = re.search(r'[a-z]+',text).group()
                        if keyword in certi_list:
                            carDic['certificate'] = keyword
                        else:
                            carDic['certificate'] = '其他'
                    else:
                        carDic['certificate'] = 'na'
                    equip=''
                    for item in soup.select('.additionConfig'):
                        equip += '{}|'.format(item.text)
                    carDic['equip'] = equip.strip('|')
                    with open(fileOut,'a') as f:
                        f.write('{}\n'.format(json.dumps(carDic, ensure_ascii=False)))
                    
                    cursor.execute('INSERT INTO cars0626 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                    (carDic['source'], carDic['url'] , carDic['title'], carDic['brand'], carDic['model'], carDic['doors'], 
                    carDic['color'], carDic['gasoline'], carDic['cc'], carDic['transmission'], carDic['equip'],
                    carDic['mileage'], carDic['years'], carDic['location'], carDic['posttime'], 
                    carDic['price'], carDic['certificate'], carDic['wd']))
                    conn.commit()  

                    break
                except Exception as e:
                    message= 'fail url={},error:{}'.format(url, e)
                    print(message)
                    with open('v3_fail_line.txt','a') as f:
                        f.write(line)
                    break
            except Exception as e:
                count+=1
                if count==3:
                    message= 'fail url={},error:{}'.format(url, e)
                    print(message)
                    with open('v3_fail_line.txt','a') as f:
                        f.write(line)
            
                
    

fileIn='v3_data_remain2.txt'
fileOut='v3_result.txt'
headers=gen_headers()
get_car_infos(fileIn,fileOut,conn,cursor,headers)


fail url=https://auto.8891.com.tw/usedauto-infos-1372404.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1380783.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1393386.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1394758.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1404852.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1404856.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1407155.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1411755.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1412486.html,error:'NoneType' object has no attribute 'text'
fail url=h

fail url=https://auto.8891.com.tw/usedauto-infos-1452680.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1399048.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1405171.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1433239.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1439369.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1443583.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1445635.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1446757.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1432043.html,error:'NoneType' object has no attribute 'text'
fail url=h

fail url=https://auto.8891.com.tw/usedauto-infos-1436319.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436322.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436326.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436332.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436335.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436340.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436508.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436526.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436536.html,error:'NoneType' object has no attribute 'text'
fail url=h

fail url=https://auto.8891.com.tw/usedauto-infos-1444182.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1435395.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1435398.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1435405.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1435408.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1435408.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1435410.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436001.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1436008.html,error:'NoneType' object has no attribute 'text'
fail url=h

fail url=https://auto.8891.com.tw/usedauto-infos-1453731.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1455383.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1406225.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1409513.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1421731.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1423319.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1423338.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1423349.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1423357.html,error:'NoneType' object has no attribute 'text'
fail url=h

...

fail url=https://auto.8891.com.tw/usedauto-infos-1398881.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1398882.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1398890.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1398896.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1398898.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1399061.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1403442.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1403443.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1403451.html,error:'NoneType' object has no attribute 'text'
fail url=h

fail url=https://auto.8891.com.tw/usedauto-infos-1450613.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1450614.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1450615.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1451120.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1451843.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1453610.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1442217.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1400809.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1407862.html,error:'NoneType' object has no attribute 'text'
fail url=h

fail url=https://auto.8891.com.tw/usedauto-infos-1453138.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1453137.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1453136.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1424280.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1424608.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1432880.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1419838.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1426330.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1430781.html,error:'NoneType' object has no attribute 'text'
fail url=h

fail url=https://auto.8891.com.tw/usedauto-infos-1398819.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1455143.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1398873.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1344015.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1368126.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1343998.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1443916.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1451082.html,error:'NoneType' object has no attribute 'text'
fail url=https://auto.8891.com.tw/usedauto-infos-1451865.html,error:'NoneType' object has no attribute 'text'
fail url=h

KeyboardInterrupt: 

In [86]:
line='https://auto.8891.com.tw/usedauto-infos-1361715.html|1490070961|{"source": "8891", "url": "https://auto.8891.com.tw/usedauto-infos-1361715.html", "title": "2009 LEXUS RX450H 總代理頂級版 原廠保養", "brand": "Lexus", "model": "RX", "cc": 3500, "transmission": "自手排", "mileage": 210000, "years": 2009, "location": "桃園市", "posttime": 1490070961, "price": 0.0}'

carDic={}
infos=line.split('\n')[0].split('|')
carDic=json.loads(infos[2])
url =infos[0]



res = requests.get('https://auto.8891.com.tw/usedauto-infos-1453226.html')
soup = BeautifulSoup(res.text,'lxml')
title=soup.select_one('.car-type').text
big_title=re.sub(r'\s+',' ',title).strip(' ')
sub_title=carDic['title']
carDic['title']='{} | {}'.format(big_title, sub_title)
carDic['color'] = soup.select('.right-info .car-info ul li span')[7].text.strip('色')
carDic['doors'] = soup.select('.car-detail-equipment .car-detail-base span')[1].text.split("：")[1]
carDic['gasoline'] = soup.select('.car-detail-equipment .car-detail-base span')[2].text.split("：")[1].strip('車')
carDic['wd'] = soup.select('.car-detail-equipment .car-detail-base span')[4].text.split("：")[1]
if soup.select('div.tip > a'):
    carDic['certificate'] = '其他'
else:
    carDic['certificate'] = 'na'
equip=''
for item in soup.select('.car-equipment-show .info'):
    equip += '{}|'.format(item.text)
carDic['equip'] = equip.strip('|')
carDic

{'brand': 'Lexus',
 'cc': 3500,
 'certificate': 'na',
 'color': '黑',
 'doors': '4門',
 'equip': '胎壓偵測|動態穩定系統|防盜系統|循跡系統|中控鎖|剎車輔助系統|兒童安全椅固定裝置|ABS防鎖死|安全氣囊|定速系統|LED頭燈|多功能方向盤|倒車雷達|電動天窗|真皮/皮革座椅',
 'gasoline': '汽油',
 'location': '桃園市',
 'mileage': 210000,
 'model': 'RX',
 'posttime': 1490070961,
 'price': 0.0,
 'source': '8891',
 'title': 'Mercedes-Benz CLA 2014款 2.0L | 2009 LEXUS RX450H 總代理頂級版 原廠保養',
 'transmission': '自手排',
 'url': 'https://auto.8891.com.tw/usedauto-infos-1361715.html',
 'wd': '2WD',
 'years': 2009}

In [70]:
carDic

{'brand': 'Honda',
 'cc': 2400,
 'certificate': 'na',
 'color': '黑',
 'doors': '4門',
 'equip': '胎壓偵測|動態穩定系統|防盜系統|循跡系統|中控鎖|剎車輔助系統|兒童安全椅固定裝置|ABS防鎖死|安全氣囊|定速系統|LED頭燈|多功能方向盤|倒車雷達|電動天窗|真皮/皮革座椅',
 'gasoline': '汽油',
 'location': '臺中市',
 'mileage': 34000,
 'model': 'CR-V',
 'posttime': 1496902979,
 'price': 58.3,
 'source': '8891',
 'title': 'Mercedes-Benz CLA 2014款 2.0L | 實價登錄 2015 CRV 4WD 頂配 免頭款全額貸 專辦信用瑕疵',
 'transmission': '手自排',
 'url': 'https://auto.8891.com.tw/usedauto-infos-1439391.html',
 'wd': '2WD',
 'years': 2015}